# Clase Extra 2 - Ensamblaje de Modelos

---

## Ensamblaje de Modelos

Es muy probable que para los problemas complejos, los clasificadores vistos anteriormente (KNN, Decision Trees, Bayes) no sean capaces de producir modelos lo suficientemente complejo como para resolverlos. 

Si bien el camino usual para resolver el problema de la complejidad es crear modelos de redes neuronales, estos por lo general requieren una gran cantidad de datos para generalizar correctamente. Por ende, si no se posee una mayor cantidad de datos, no siempre será la elección correcta.

Una opción interesante para tener en cuenta es el de utilizar **ensamblaje de modelos**. 

Los modelos generados por ensamblaje no son otra variedad de modelos más complejos y distintos a los vistos anteriormente, si no que son frameworks para generar **meta-modelos**, que no son otra cosa que combinaciones de muchos modelos poco complejos y con poca precisión (*weak-learners*).
La hipótesis principal de estos modelos es que la combinación de varios *weak-learners* poco precisos permite generar modelos mucho más complejos con la misma cantidad de datos.

Los *weak learners* pueden ser entrenados con pocos datos y muy rápidamente; tal como caso de los árboles de decisión. 
La combinación de estos modelos poco complejos se hace a través de algún mecanismo de agregación/votación. 



Existen dos paradigmas de creación de modelos ensamblados: **Bagging y Boosting**.

### Bagging (Bootstrap Aggregating)

**Bagging** consiste en entrenar varios *weak learners* (ej, trees) usando distintos muestreos del dataset de entrenamiento y luego combinar los modelos entrenados.


<br>

<div align='center'>
    <img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/21_Ensamblaje/ensemble_bagging.png?raw=true' width=800/>
</div>

<br>
<div align='center'>
    Fuente: <a href='https://en.wikipedia.org/wiki/Bootstrap_aggregating'>Boosting en Wikipedia</a>
</div>
<br>



Un ejemplo clásico de este es `RandomForest`.
Su funcionaimento es relativamente sencillo: 

Supongamos que se entrenarán $B$ ($B$ de bags) árboles de decisión y que cada arbol será entrenado con $n$ datos. Entonces,



- Por cada $b=1,...B$:
    - Obtenemos un subconjunto de entrenamiento con n elementos a través de un muestreo con reemplazo (*bootstrap*) sobre conjunto de entrenamiento original.
    - Entrenamos un árbol de decisión para regresión o clasificación $f_b$ usando ese set.

Luego, dado un ejemplo por predecir $x$, para obtener los valores de:

- Una regresión se calcula: 

$$\hat{y} = \frac{1}{B}\sum_{b=1}^{B}f_b(x)$$

- Una clasificación, se retorna la clase con más votos. 


EL siguiente ejemplo consiste en un problema de detección de fraudes bancarios. 
Para mostrar el funcionamiento de Random Forest, evaluaremos la tarea anterior usando un modelo `DecisionTree` y compararemos el aumento de rendimiento usando luego un `RandomForest`

## Credit Card Fraud

El problema de esta clase consiste en, dados ciertos atributos preprocesados usando PCA (por lo que no sabemos que codifica cada dimensión), generar un clasificador que prediga si una fila representa o no un fraude bancario.

La fuente de los datos la pueden encontrar en [creditcardfraud en Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud).

In [ ]:
import pandas as pd

df = pd.read_pickle('/Users/imezadelajara/Repositories/MDS7202/recursos/2023-01/21_Ensamblaje/creditcard.pickle')
df = df.drop(columns=['Time'])
df

#### Desablance de Clases

Una de las cosas que pueden notar de este dataset es un tremendo desbalance de clases: 284315 no fraudes vs 492 fraudes!

In [ ]:
# normalise the amount column
features = df.drop(columns=['Class']) 
labels = df['Class']
# as you can see there are 492 fraud transactions.
labels.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    shuffle=True, 
                                                    stratify=labels,
                                                    test_size=0.3,
                                                    random_state=10)

#### Pipeline con DecisionTree

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix

# generar el pipeline
pipe1 = Pipeline([('Preprocesamiento',
                   ColumnTransformer([
                       ('Scaler', StandardScaler(), ['Amount']),
                   ],
                                     remainder='passthrough')),
                  ('Decision Tree', DecisionTreeClassifier(random_state=44))])

# entrenar
pipe1.fit(X_train, y_train.ravel())

# predecir
y_pred = pipe1.predict(X_test)

# reporte de clasificación
print('Matriz de confusión\n')
print(confusion_matrix(y_test, y_pred))
print('\nReporte de Clasificación\n')
print(classification_report(y_test, y_pred))

#### Pipeline Con RandomForest

In [ ]:
# generar el pipeline
pipe2 = Pipeline(
    [('Preprocesamiento', ColumnTransformer([
        ('Scaler', StandardScaler() , ['Amount']),
    ], remainder='passthrough')),
     ('Random Forest', RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=44))
    ])

# entrenar
pipe2.fit(X_train, y_train.ravel())
 
# predecir
y_pred = pipe2.predict(X_test)

# reporte de clasificación
print('Matriz de confusión\n')
print(confusion_matrix(y_test, y_pred))
print('\nReporte de Clasificación\n')
print(classification_report(y_test, y_pred))

Como se puede ver, `RandomForest` incremento en la precisión y recall de la clase 1 con respecto al `DecisionTree`

### Boosting

El segundo enfoque para crear modelos ensamblados es Boosting. En este caso, cada modelo se entrena con todos los datos, pero los modelos se entrenan de forma iterativa de tal forma que los modelos siguientes intentan corregir los errores de los modelos anteriores.

<br>

<div align='center'>
    <img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/21_Ensamblaje/ensemble_boosting.png?raw=true' width=800/>
</div>

<br>
<div align='center'>
    Fuente: <a href='https://en.wikipedia.org/wiki/Boosting_(machine_learning)'>Boosting en Wikipedia</a>
</div>
<br>

La idea, en términos simples, es que los primeros modelos predigan bien los casos más generales, mientras que los modelos más profundos se encarguen de predecir los casos más particulares y dificiles.



El modelo ensamblado final será una combinación de los modelos creados en el proceso iterativo. Uno de los algoritmos más populares para crear este tipo de modelos es `GradientBoosting` (usaremos `HistGradientBoostingClassifier`, una implementación mucho más rápida en el ejemplo). Una explicación de su idea principal viene a continuación.


Dada $M$ iteraciones, el algoritmo para generar un modelo para regresión es:

- Comenzamos con un modelo constante $f = f_0$
- Luego, por cada label $i=1,\dots,N$, la transformamos a $\hat{y}_i \leftarrow y_i - f(x_i)$. $\hat{y}_i$ es llamado residual y su conjunto se transforma en las nuevas labels de $x$.
- Ahora, entrenamos un nuevo modelo (tree) $f_1$ y redefinimos $f = f_0 + \alpha f_1$. $\alpha$ es conocido como learning rate.
- Computamos nuevamente los residuales y calculamos un nuevo modelo $f_2$. Luego, fijamos el modelo predictivo como $f = f_0 + \alpha f_1 + \alpha f_2$.
- Repetimos hasta que se cumplan $M$ iteraciones.


La idea general es que en cada iteración los residuales más cercanos a 0 son los que se están clasificando correctamente mientras que los residuales con valores más altos son mal predichos se espera que se vayan corrigiendo en los modelos más profundos hasta alcanzar el modelo $M$.

El caso de la clasificación es un poco más complejo y utiliza modelos de Regresión Logística para lograrlo. Queda propuesta su profundización.

#### Pipeline con GradientBoosting

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

# generar el pipeline
pipe3 = Pipeline(
    [('Preprocesamiento', ColumnTransformer([
        ('Scaler', StandardScaler() , ['Amount']),
    ], remainder='passthrough')),
     ('HistGradientBoostingClassifier', HistGradientBoostingClassifier(
         random_state=44, 
         l2_regularization=10, 
         max_depth=15,
         max_iter=1000))
    ])

# entrenar
pipe3.fit(X_train, y_train.ravel())
 
# predecir
y_pred = pipe3.predict(X_test)

# reporte de clasificación
print('Matriz de confusión\n')
print(confusion_matrix(y_test, y_pred))
print('\nReporte de Clasificación\n')
print(classification_report(y_test, y_pred))

Nuevamente vemos mejores resultados en este tipo de clasificador con respecto al árbol básico. 


En general, Gradient Boosting puede ser más poderoso que los basados en bagging, pero si entrenamiento de caracter secuencial puede hacelo mucho más lento.

**Referencias**


- [1] The Hundred-Page Machine Learning Book, Andriy Burkov. Artículo 7.5